# Get GFS data from the source
- Data dictionary: https://rda.ucar.edu/datasets/ds084.1/#metadata/grib2.html?_do=y
- Data access: https://rda.ucar.edu/datasets/ds084.1/#!access

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import httpx
import xarray as xr
from dotenv import load_dotenv

from download import download_file

In [ ]:
load_dotenv()

## Authenticate and get cookie

In [ ]:
auth_url = "https://rda.ucar.edu/cgi-bin/login"
auth_data = {
    "email": os.environ["UCAR_EMAIL"],
    "passwd": os.environ["UCAR_PASS"],
    "action": "login",
}
res = httpx.post(auth_url, data=auth_data)
assert res.status_code == 200
cookies = res.cookies

## Build path and download

In [ ]:
year = "2022"
month = "08"
day = "13"
hour = "00"
fnum = "f000"

ymd = f"{year}{month}{day}"
data_url = f"https://rda.ucar.edu/data/ds084.1/{year}/{ymd}/gfs.0p25.{ymd}{hour}.{fnum}.grib2"
data_url

In [ ]:
file = download_file(data_url, cookies=cookies)

## Load into xarray

Possible values for `typeOfLevel`:
- meanSea
- hybrid
- atmosphere
- surface
- planetaryBoundaryLayer
- isobaricInPa
- isobaricInhPa
- heightAboveGround
- depthBelowLandLayer
- heightAboveSea
- atmosphereSingleLayer
- lowCloudLayer
- middleCloudLayer
- highCloudLayer
- cloudCeiling
- heightAboveGroundLayer
- tropopause
- maxWind
- isothermZero
- highestTroposphericFreezing
- pressureFromGroundLayer
- sigmaLayer
- sigma
- potentialVorticity

In [ ]:
ds = xr.open_dataset(
    file,
    engine="cfgrib",
    backend_kwargs={"filter_by_keys": {"typeOfLevel": "sigma"}},
)
ds